# Operaciones con ventanas

Spark Streaming proporciona las siguientes operaciones con ventanas deslizantes sobre los datos. Los parámetros más importantes a la hora de definir la ventana son los siguientes:

* window length - Duración de la ventana en segundos.
* sliding interval - Intervalo de desplazamiento de la ventana en segundos.

Ambos parámetros han de ser múltiplos de la duración del batch del DStream original.


| Método        | Funcionamiento           |
| -------------:|:-------------|
| **window**(windowLength, slideInterval)      | Nuevo DStream calculado según batches en ventanas del DStream original, tamaño de ventana *windowLength* y salto entre ventanas *slideInterval*. |
| **countByWindow**(windowLength, slideInterval)     | Devuelve el conteo de elementos de la ventana deslizante definida con *windowLength* y *slideInterval*, sobre el DStream original.    |
| **reduceByWindow**(func, windowLength, slideInterval) | Similar a reduce, permite la agregación de elementos de cada RDD del DStream original para generar un nuevo DStream con RDDs simples, sobre los batches de la ventana deslizante definida con *windowLength* y *slideInterval*. La función *func* ha de ser conmutativa y asociativa, recibir dos argumentos y devolver uno.     |
| **reduceByKeyAndWindow**(func, windowLength, slideInterval, [numTasks])     | Similar a reduceByKey, sobre un Dstream original con pares clave-valor (K,V), devuelve el agregado según la función *func* para cada clave, sobre los *batches* de la ventana deslizante definida con *windowLength* y *slideInterval*. Número de tareas paralelas opcional.  |
| **countByValueAndWindow**(windowLength, slideInterval, [numTasks]) | Sobre un DStream de pares clave-valor (K,V), devuelve la frecuencia de cada clave en formato (K, Long), sobre los *batches* de la ventana deslizante definida con *windowLength* y *slideInterval*. Número de tareas paralelas opcional.      |

### Demo

In [1]:
import findspark
#Importante: Modificar la ruta para que apunte al HOME de Spark
findspark.init('/opt/spark')

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [3]:
sc = SparkContext(master="local[2]", appName="WindowWordcount")
ssc = StreamingContext(sc, 1)

#El uso de checkpoints es necesario cuando se van a realizar operaciones con ventanas
ssc.checkpoint("checkpoint")

### Introducción de datos
Se va a utilizar un socket para introducir datos. Dicho socket estará escuchando en el puerto 9999 de localhost. Para la introducción de datos se puede abrir una ventana de comandos y ejecutar el comando "nc -lk 9999", que abre el puerto y lo mantiene abierto, para a continuación pegar los mensajes que se quieren enviar. Si no existe el comando, deberemos instalar el programa "netcat" --> "sudo apt-get install netcat"

In [4]:
lines = ssc.socketTextStream("localhost", 9999)

Se van a probar tres usos de ventanas distintas (recomendable sólo ejecutar una transformación cada vez para evitar comportamientos erróneos, es decir, comentar las otras dos transformaciones):
- En la primera, se muestran las palabras que aparecen en cada ventana, sin agrupar (se repiten las palabras).
- En la segunda, se cuenta el número de palabras totales en cada ventana (countByWindow).
- En la tercera, se muestra el número de veces que aparece cada palabra en cada ventana (reduceByKeyAndWindow).

Introducir cualquier combinación de letras, palabras o frases, dejando un tiempo entre una frase y otra. Para comprobar la tercera transformación, procurar que en la misma ventana de 5 segundos haya palabras que aparezcan más de una vez.

In [5]:
words = lines.flatMap(lambda line: line.strip().split(" "))
pairs = words.map(lambda word: (word, 1))

#Las ventanas muestran los valores correspondientes a los últimos 5 segundos (con un desplazamiento de un segundo).
pairs.window(5, 1).pprint()
pairs.countByWindow(5, 1).pprint()
pairs.reduceByKeyAndWindow(lambda x,y:(x+y), 5, 1).pprint()


In [ ]:
ssc.start()

In [ ]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)